In [2]:
from PIL import Image, ImageFont, ImageDraw
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
import csv
import cv2
from cv2 import __version__
print __version__

3.2.0


In [3]:
def find_catchlights(image, threshold):
    #thresholding
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,thresh1 = cv2.threshold(gray,threshold,255,cv2.THRESH_BINARY)
    #we use erosion and dilation to remove noise
    morphKernel = np.ones((10, 10), np.uint8)
    morph = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, morphKernel)
    #find contours
    _, contours, hierarchy = cv2.findContours(morph.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #find catchlights using hierarchy
    largest = [1,2]
    catchlights = []
    for i in range(3,len(contours)):
        if (cv2.contourArea(contours[i]) > cv2.contourArea(contours[largest[0]])):
            largest[1] = largest[0]
            largest[0] = i
        elif (cv2.contourArea(contours[largest[0]]) > cv2.contourArea(contours[i]) > cv2.contourArea(contours[largest[1]])):
            largest[1] = i
    for i in range(0,len(contours)):
        if (hierarchy[0][i][3] == largest[0])or(hierarchy[0][i][3] == largest[1]):
            catchlights.append(contours[i])
    pupils = [contours[largest[0]],contours[largest[1]]]
    return catchlights, pupils

In [4]:
def get_brightness(image,contours):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = np.zeros(gray.shape,np.uint8)
    cv2.drawContours(mask,contours,-1,255,-1)
    pixelpoints = np.transpose(np.nonzero(mask))
    return cv2.mean(gray,mask = mask)[0],pixelpoints.size

In [5]:
def check_blink(catchlights):
    #simple method
    if (len(catchlights) <= 2) or (len(catchlights) >= 6):
        return True
    return False

In [7]:
video_name = "2012-07-16-13.25.47_1"

out = cv2.VideoWriter('videos/' + video_name + "_out.avi",cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1920,1080))
cap = cv2.VideoCapture('videos/' + video_name + ".mov")

for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    success,frame = cap.read()
    if success == False: break
    catchlights,pupils = find_catchlights(frame,45)
    #draw catchlights
    cv2.drawContours(frame, catchlights, -1, (0, 255, 0), 5)
    #draw iris, in red when blinking
    if check_blink(catchlights): cont_color = (0,0,225)
    else: cont_color = (225,0,0)
    cv2.drawContours(frame, pupils, -1, cont_color, 5)
    #write brightness
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,str(i) + ", " + str(get_brightness(frame,catchlights)),(10,900), font, 1,(255,255,255),2,cv2.LINE_AA)
    #write to output
    out.write(frame)

cap.release()
out.release()
    
        